In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
n = 10
x = np.arange(n)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [68]:
def chunk_spectrogram(chunk, nperseg=4, noverlap=1, rate=44100, filename='spectrogram'):
    '''
    v1.0 only window
    v2.0 without blur/sharp/gauss kernels <--- now makin this
    v2.1 multi track support / multi / instrument
    v3.0 with blur/sharp/gauss
    v4.0 cupy support
    '''
#     chunk_n = len(chunk)
    step = nperseg - noverlap
    shape = chunk.shape[:-1] + ((chunk.shape[-1] - noverlap) // step, nperseg)
    strides = chunk.strides[:-1] + (step * chunk.strides[-1], chunk.strides[-1])
    chunk_rolled = np.lib.stride_tricks.as_strided(chunk, shape=shape, strides=strides)
    
    f_N = nperseg // 1
    t_chunk = np.linspace(0, nperseg / rate, nperseg)
    f = np.logspace(np.log10(30), np.log10(nperseg//2), f_N) # second arg: nperseg//2 (Nyquist limit)
    # f = np.geomspace(30, nperseg//2, f_N) # second arg: nperseg//2 (Nyquist limit)
    
    T_chunk, F_chunk = np.meshgrid(t_chunk, f)
    E = np.array(np.exp(-2j * np.pi * F_chunk * T_chunk))
    
    # chunk_rolled = chunk_rolled * window
    
    corr = np.dot(chunk_rolled, E.T)
    magnitude = np.abs(corr)
    
    spectrogram = magnitude.T
    

    with open(f'{filename}.png', 'wb') as file:
        plt.imsave(file, spectrogram, cmap='viridis', origin='lower')
    os.system(f'qlmanage -p {filename}.png')
#     files.download('spectrogram.png')
    print(magnitude)

In [69]:
# nperseg  = 11025 // 5
# noverlap = nperseg // 100 * 98

chunk_spectrogram(x, rate=3)

[[ 1.73205081  1.73051558  3.89868938  5.22480623]
 [ 4.58257569  4.44308725  6.31170243 13.28661239]
 [ 7.54983444  7.29614313  9.43997907 21.79460957]]


In [35]:
E = np.arange(20).reshape(5,4)
E

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [18]:
np.dot(E, np.array([0, 1, 2, 3]))

array([14, 38, 62, 86])

In [21]:
np.dot(E, np.array([3, 4, 5, 6]))

array([ 32, 104, 176, 248])

In [22]:
np.dot(E, np.array([6, 7, 8, 9]))

array([ 50, 170, 290, 410])

In [33]:
t = np.array(
    [[0, 1, 2, 3],
     [3, 4, 5, 6],
     [6, 7, 8, 9]]
)

In [36]:
print(np.dot(E, t[0]))
print(np.dot(E, t[1]))
print(np.dot(E, t[2]))

[ 14  38  62  86 110]
[ 32 104 176 248 320]
[ 50 170 290 410 530]


In [40]:
# np.dot(E, z)
np.dot(E, t.T)

array([[ 14,  32,  50],
       [ 38, 104, 170],
       [ 62, 176, 290],
       [ 86, 248, 410],
       [110, 320, 530]])

In [42]:
np.dot(t, E.T)

array([[ 14,  38,  62,  86, 110],
       [ 32, 104, 176, 248, 320],
       [ 50, 170, 290, 410, 530]])

# SIMPLE ROLLING WINDOW [STRIDES]

In [3]:
n = 10
x = np.arange(n)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
nperseg  = 4
noverlap = 1
step = nperseg - noverlap
shape = ((x.shape[-1] - noverlap) // step, nperseg)
strides = (step * x.strides[-1], x.strides[-1])
result = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)
print(result)

[[0 1 2 3]
 [3 4 5 6]
 [6 7 8 9]]
